In [3]:
import nltk
import os
import re
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from tqdm import tqdm

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/dnlab/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
def preprocess(doc):
    # Remove punctuations and numbers
    doc = re.sub(r'[^\w\s]', '', doc)
    doc = re.sub(r'\d+', '', doc)
    # Normalization
    doc = doc.lower()
    # Tokenization
    tokens = doc.split()
    # Stop words removal
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word not in stop_words]
    # Stemming
    ps = PorterStemmer() 
    terms = []
    for word in filtered_tokens:
        terms.append(ps.stem(word))
    doc = ' '.join(terms)
    return doc


def build_inverted_index(dir_path):
    inverted_index = {}
    doc_id = 0
    for dir, _, filename in os.walk(dir_path):
        for file in tqdm(filename):
            # The script crashes processing .lws files
            if file.endswith('.swp'):
                continue
            with open(os.path.join(dir, file), 'r', encoding='latin-1') as f:
                # print(f"processing: {f}")
                doc = f.read()
            doc = preprocess(doc)
            # Split the document into terms
            terms = doc.split()
            for term in terms:
                if term not in inverted_index:
                    inverted_index[term] = []
                if doc_id not in inverted_index[term]:
                    inverted_index[term].append(doc_id)
            doc_id += 1
    return inverted_index


In [5]:
dataset = '/home/dnlab/Desktop/stories'
inverted_index = build_inverted_index(dataset)

100%|██████████| 19/19 [00:00<00:00, 22.74it/s]


In [6]:
import json

with open('inverted_index.json', 'w') as f:
    json.dump(inverted_index, f)


In [7]:
# load the inverted index from the json file to avoid running the entire script
with open('inverted_index.json', 'r') as f:
    inverted_index = json.load(f)


In [8]:
def boolean_query_and(term1, term2, inverted_index):
    docs_list1 = set(inverted_index[term1]) if term1 in inverted_index else set()
    docs_list2 = set(inverted_index[term2]) if term2 in inverted_index else set()
    result = docs_list1.intersection(docs_list2)
    return result


term1 = 'ben'
term2 = 'june'

and_query = boolean_query_and(term1=term1, term2=term2, inverted_index=inverted_index)
print(and_query)

{0, 298, 149, 407, 410}


In [9]:
def boolean_query_or(term1, term2, inverted_index):
    docs_list1 = set(inverted_index[term1]) if term1 in inverted_index else set()
    docs_list2 = set(inverted_index[term2]) if term2 in inverted_index else set()
    result_docs = docs_list1.union(docs_list2)
    return result_docs


term1 = 'ben'
term2 = 'june'

or_query = boolean_query_or(term1=term1, term2=term2, inverted_index=inverted_index)
print(or_query)


{0, 128, 141, 17, 149, 21, 407, 151, 410, 284, 290, 36, 422, 167, 298, 53, 186, 315, 59, 66, 327, 74, 75, 81, 85, 344, 225, 109, 237, 377, 123, 125, 126}


In [10]:
def boolean_query_and_not(term1, term2, inverted_index):
    docs_list1 = set(inverted_index[term1]) if term1 in inverted_index else set()
    docs_list2 = set(inverted_index[term2]) if term2 in inverted_index else set()
    result_docs = docs_list1.difference(docs_list2)
    return result_docs


term1 = 'ben'
term2 = 'june'

and_not_query = boolean_query_and_not(term1=term1, term2=term2, inverted_index=inverted_index)
print(and_not_query)


{128, 225, 66, 36, 75, 123, 109, 141, 17, 85, 344, 315}


In [12]:
def boolean_query_or_not(term1, term2, term3, inverted_index):
    # Retrieve the document IDs for each term in the query
    docs_list1 = set(inverted_index[term1]) if term1 in inverted_index else set()
    docs_list2 = set(inverted_index[term2]) if term2 in inverted_index else set()
    docs_list3 = set(inverted_index[term3]) if term3 in inverted_index else set()
    # Compute the union of the document IDs for term1 and term2, and subtract the document IDs for term3
    result_docs = docs_list1.union(docs_list2) - docs_list3
    return result_docs


term1 = 'ben'
term2 = 'adler'
term3 = 'june'
or_not_query = boolean_query_or_not(term1, term2, term3, inverted_index)
print(or_not_query)


{128, 225, 66, 36, 75, 141, 109, 17, 315, 85, 344, 123}


In [14]:
# Ranked retrieval
from collections import Counter
from math import log10

def taat_ranked_retrieval(query, inverted_index, k=10):
    # Tokenize the query
    query_terms = preprocess(query).split()
    scores = {}
    for term in query_terms:
        if term not in inverted_index:
            continue
        # Retrieve postings list for the term
        postings = inverted_index[term]
        # Compute TF scores for each document
        tf_scores = Counter(postings)
        # Compute IDF score for the term
        idf_score = log10(len(inverted_index) / len(postings))
        # Compute the product of TF and IDF scores for each document
        for doc_id in tf_scores:
            if doc_id not in scores:
                scores[doc_id] = 0
            scores[doc_id] += tf_scores[doc_id] * idf_score
    # Sort documents by their scores in descending order
    ranked_docs = sorted(scores.items(), key=lambda x: x[1], reverse=True)[:k]
    return [doc_id for doc_id, score in ranked_docs]


In [17]:
taat_retrieval = taat_ranked_retrieval(query='rattling through the silent streets', inverted_index=inverted_index)

print(taat_retrieval)

[1, 5, 17, 38, 53, 60, 63, 78, 114, 128]
